In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from scipy.stats import poisson, norm

In [ ]:
norm.rvs(1,1,10)

In [ ]:
n = int(1e6)

mu_demand = 65
mu_supply = mu_demand*1.03
mu_supply_2 = mu_supply*1.03

np.random.seed(5)

demand_seq = np.random.poisson(mu_demand,n)
supply_seq = np.random.poisson(mu_supply,n)
supply_2_seq = np.random.poisson(mu_supply_2,n)

#demand_seq = norm.rvs(mu_demand,np.sqrt(mu_demand),n)
#supply_seq = norm.rvs(mu_supply,np.sqrt(mu_supply),n)

In [ ]:
def simulate_reflected_random_walk(arrival_sequence,transfer_sequence,drain_sequence,thres):
    arrival_buffer = np.zeros(len(arrival_sequence)+1)
    drain_buffer = np.zeros(len(arrival_sequence)+1)
    arrival_buffer[0] = 0#5e5
    drain_buffer[0] = 5e5
    n_idles = 0
    idling_benefit = 0
    idling_benefit_2 = 0
    for i,(a,t,d) in enumerate(zip(arrival_sequence,transfer_sequence,drain_sequence)):
        t = min(t,a+arrival_buffer[i])
        if drain_buffer[i] > thres:
            t = 0
            n_idles += 1
            idling_benefit += a-d
            idling_benefit_2 += a-min(d,drain_buffer[i])
        arrival_buffer[i+1] = max(0,arrival_buffer[i]+a-t)
        drain_buffer[i+1] = max(0,drain_buffer[i]+t-d)
        
    print(n_idles,idling_benefit,idling_benefit_2)
    return arrival_buffer,drain_buffer

In [ ]:
c1 = 1
c2 = 1.5

In [ ]:
arrival_buffer,drain_buffer = simulate_reflected_random_walk(demand_seq,supply_seq,supply_2_seq,mu_supply_2*3)

In [ ]:
plt.plot(drain_buffer[:700]*c2+arrival_buffer[:700]*c1)

In [ ]:
(mu_supply_2-mu_demand)/(mu_supply-mu_demand)

In [ ]:
plt.plot(hedging,cost)
plt.plot(hedging,cost,".")

In [ ]:
25*np.sqrt(mu_supply)/mu_supply

In [ ]:
plt.plot(hedging,cost)
plt.plot(hedging,cost,".")

In [ ]:
plt.plot(hedging,cost)
plt.plot(hedging,cost,".")

In [ ]:
cost[1]/cost[-1]

In [ ]:
mu_supply_2*6

In [ ]:
hedging

In [ ]:
len(demand_seq)

In [ ]:
cost = []
hedging = np.arange(15,40,5)
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(8,8))
#plot_range = range(4000,5000)
plot_range = range(0,50000)
cumul = False
for h in hedging:
    thres = mu_supply_2+h*np.sqrt(mu_supply_2)
    #thres = h*10
    print(thres)
    arrival_buffer,drain_buffer = simulate_reflected_random_walk(demand_seq,supply_seq,supply_2_seq,thres)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
f.legend()
#plt.plot(hedging,cost)

In [ ]:
cost[0]-cost[1]

In [ ]:
plt.plot(drain_buffer[:300])

In [ ]:
plt.plot(arrival_buffer[:600])

In [ ]:
plt.plot(buffer_seq[:1000])

In [ ]:
sum(buffer_seq)

In [ ]:
sum(buffer_seq)

In [ ]:
np.percentile((supply_seq-demand_seq)[(supply_seq-demand_seq) < 0],0.01)

In [ ]:
plt.plot(np.cumsum(supply_seq)-np.cumsum(demand_seq))

In [ ]:
percentile = 1.645
#percentile = 0
#percentile = 1.96
#percentile = 2.33
slack_capacity = mu_supply-mu_demand
time_horizon = (percentile**2)*mu_supply/(2*slack_capacity**2)
offset = time_horizon*(-slack_capacity) + percentile* np.sqrt(mu_supply*2*time_horizon)
print(time_horizon*2)
time_horizon = int(np.ceil(time_horizon))
offset = int(np.ceil(offset))

In [ ]:
time_horizon = (percentile**2)*mu_supply*2/slack_capacity**2
time_horizon = int(np.ceil(time_horizon))

In [ ]:
y = []
for d in range(time_horizon):
    y.append(d*(slack_capacity) - percentile* np.sqrt(mu_supply*2*d))

In [ ]:
y_1 = y
time_horizon_1 = time_horizon

In [ ]:
y_2 = y
time_horizon_2 = time_horizon

In [ ]:
time_horizon/time_horizon_1

In [ ]:
1.96/1.645

In [ ]:
plt.plot(range(time_horizon),y)
plt.plot(range(time_horizon_1),y_1)
plt.plot(range(time_horizon_2),y_2)

In [ ]:
y

In [ ]:
time_horizon

In [ ]:
offset

In [ ]:
thres = poisson.ppf(0.95,mu_demand)
#thres = 0
thres = poisson.ppf(0.5,mu_demand)

def idle_supply(demand_seq,supply_seq,offset):
    inv_pos = offset
    idle_supply_seq = np.zeros_like(supply_seq) 
    idle_count = 0
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        if inv_pos > thres+offset:
            s = 0
            idle_count += 1
        idle_supply_seq[i] = s
        inv_pos += s-d
    #print(idle_count/len(supply_seq)) 
    return idle_supply_seq

def idle_supply_time_horizon(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        if (inv_pos > thres+offset) and days_count >= time_horizon:
            s = 0
            days_count = 0
        idle_supply_seq[i] = s
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
        days_count += 1
    return inv_pos_seq

def idle_supply_time_horizon_smooth(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    just_idled = False
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0 and ((days_count >= time_horizon) or just_idled):
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            days_count=0
            just_idled = True
        else:
            just_idled = False
            
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
        if not just_idled:
            days_count += 1
            
    return inv_pos_seq

def work_supply_time_horizon_smooth(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    just_idled = True
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0 and ((days_count >= time_horizon) or just_idled):
            days_count = 0
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            days_count=0
            just_idled = True
        else:
            days_count += 1
            just_idled = False
            
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
            
    return inv_pos_seq

def idle_supply_smooth(demand_seq,supply_seq,offset):
    inv_pos = offset
    idle_supply_seq = np.zeros_like(supply_seq) 
    idle_count = 0
    inv_pos_array = np.zeros_like(supply_seq)
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0:
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            idle_count += 1
            
        idle_supply_seq[i] = s
        inv_pos += s-d
        inv_pos = min(inv_pos,offset)
        inv_pos_array[i] = inv_pos

    #print(idle_count/len(supply_seq)) 
    print(inv_pos)
    return inv_pos_array

In [ ]:
slack_capacity/np.sqrt(2*mu_demand)

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+500])

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+500])

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+100])

In [ ]:
offset

In [ ]:
time_horizon*slack_capacity/2

In [ ]:
slack_capacity

In [ ]:
inv_pos_seq = work_supply_time_horizon_smooth(demand_seq,supply_seq,53,12)

print(np.mean(inv_pos_seq < 0))

In [ ]:
inv_pos_seq = idle_supply_time_horizon_smooth(demand_seq,supply_seq,53,12)

print(np.mean(inv_pos_seq < 0))

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
inv_pos_seq = idle_supply_time_horizon_smooth(demand_seq,supply_seq,41,69)

print(np.mean(inv_pos_seq < 0))

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
inv_pos_seq = idle_supply_time_horizon(demand_seq,supply_seq,offset,time_horizon)

print(np.mean(inv_pos_seq < 0))
#plt.plot(inv_pos_seq[827341-10:827341+10])
#plt.plot(inv_pos_seq[827341-10:827341+10],".")

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
idle_supply_seq,inv_pos_seq = idle_supply_smooth(demand_seq,supply_seq, np.ceil(offset))
#inv_pos_seq = offset + np.cumsum(idle_supply_seq)-np.cumsum(demand_seq)
print(np.mean(inv_pos_seq < 0))
#plt.plot(inv_pos_seq[827341-10:827341+10])
#plt.plot(inv_pos_seq[827341-10:827341+10],".")

plt.plot(inv_pos_seq[:1200])

In [ ]:
n_sims = 100000
demand_sum = np.random.poisson(mu_demand*np.ceil(time_horizon),n_sims)
supply_sum = np.random.poisson(mu_supply*np.ceil(time_horizon),n_sims)

print(np.mean((demand_sum-supply_sum) > np.ceil(offset)))

In [ ]:
offset+time_horizon*slack_capacity

In [ ]:
1001 % 100

In [ ]:
offset

In [ ]:
time_horizon*slack_capacity/2

In [ ]:
np.random.seed(500)
n_sims = 100000
#n_sims = 20
stockouts = []
last_day_stockouts = []
last_day_stockouts_vals = []
ave_inventories = []
sim_time_horizon = time_horizon
for i in range(n_sims):
    demand = np.random.poisson(mu_demand,sim_time_horizon)
    supply = np.random.poisson(mu_supply,sim_time_horizon)
    inv_pos_seq = offset + np.cumsum(supply)-np.cumsum(demand)
    stockouts.append(np.sum(inv_pos_seq < 0))
    last_day_stockouts.append(inv_pos_seq[-1] < offset)
    if last_day_stockouts[-1]:
        last_day_stockouts_vals.append(inv_pos_seq[-1]-offset)
    
    ave_inventories.append(np.mean(inv_pos_seq))
    if i % 10000 == 0:
        plt.plot(inv_pos_seq)
    
sum(stockouts)/(sim_time_horizon*n_sims),np.sum(last_day_stockouts)/(n_sims),np.mean(ave_inventories)


In [ ]:
offset

In [ ]:
np.median(last_day_stockouts_vals)

In [ ]:
for offset in range(200):
    stock_out_probs = []
    for d in range(1,time_horizon+1):
        stock_out_prob = norm.cdf(-offset,slack_capacity*d,np.sqrt(2*mu_supply*d))
        stock_out_probs.append(stock_out_prob)
    overal_stockout_prob = np.mean(stock_out_probs)
    #print(overal_stockout_prob)
    if overal_stockout_prob < 0.05:
        break

In [ ]:
time_horizon

In [ ]:
def get_percentile_deficit(cycle_dur,slack_capacity,variance,percentile = 0.5):
    mu = slack_capacity*cycle_dur
    std = np.sqrt(variance*cycle_dur)
    cum_deficit_prob = norm.cdf(0,mu,std)
    cum_percentile = 0
    prev_cum_prob = cum_deficit_prob
    for i in range(10000):
        cum_prob = norm.cdf(-i,mu,std)
        prob = (prev_cum_prob - cum_prob)/cum_deficit_prob
        cum_percentile += prob
        if cum_percentile >= percentile:
            return i
        prev_cum_prob = cum_prob
        
a = get_percentile_deficit(time_horizon/4,slack_capacity,2*mu_supply)
#get_percentile_deficit(slack_capacity,2*mu_supply,time_horizon)
print(a)

def compute_recovery_time(slack_capacity,variance,deficit,bound = 2.33):
    dur = ((bound*np.sqrt(variance)+np.sqrt(bound**2*variance+4*slack_capacity*deficit))/(2*slack_capacity))**2
    return int(np.ceil(dur))

print(compute_recovery_time(slack_capacity,2*mu_supply,a))

def get_average_stockout_prob(duration,slack_capacity,variance,start):
    stock_out_probs = []
    for d in range(1,duration+1):
        stock_out_prob = norm.cdf(0,start+slack_capacity*d,np.sqrt(variance*d))
        stock_out_probs.append(stock_out_prob)
    average_stockout_prob = np.mean(stock_out_probs)
    return average_stockout_prob

def compute_stockout_prob_and_inventory_cost(cycle_dur,slack_capacity,variance,offset):
    mu = slack_capacity*cycle_dur
    std = np.sqrt(variance*cycle_dur)
    cum_deficit_prob = norm.cdf(0,mu,std)
    #print(cum_deficit_prob)
    deficit = get_percentile_deficit(cycle_dur,slack_capacity,variance,0.95)
    #print(deficit)
    rec_dur = compute_recovery_time(slack_capacity,variance,deficit)
    #print(rec_dur)
    cycle_stockout_prob = get_average_stockout_prob(cycle_dur,slack_capacity,variance,offset)
    
    rec_dur = int(np.ceil(deficit/slack_capacity))
    print(rec_dur)
    rec_stockout_prob = get_average_stockout_prob(rec_dur,slack_capacity,variance,offset-deficit)
    #print(cycle_stockout_prob,rec_stockout_prob)
    
    effective_duration = (cycle_dur+cum_deficit_prob*rec_dur)
    #print(cycle_dur/effective_duration)
    
    overall_stockout_prob = (cycle_dur*cycle_stockout_prob+cum_deficit_prob*rec_dur*rec_stockout_prob)/effective_duration
    
    overall_inventory_cost = (cycle_dur*(0.5*slack_capacity*cycle_dur+offset)+cum_deficit_prob*rec_dur*(0.5*slack_capacity*rec_dur+offset-deficit))/effective_duration
    #print(overall_inventory_cost)
    return overall_stockout_prob,overall_inventory_cost

In [ ]:
time_horizon/4

In [ ]:
variance = 2*mu_supply

In [ ]:
min_inv_cost = np.inf
min_cycle_dur = None
min_offset = None
for cycle_dur in range(1,int(time_horizon)):

    for offset in range(200):
        overall_stockout_prob,inv_cost = compute_stockout_prob_and_inventory_cost(cycle_dur,slack_capacity,variance,offset)
        #print(overall_stockout_prob)
        if overall_stockout_prob < 0.05:
            break
    
    print(cycle_dur,inv_cost)
    if inv_cost < min_inv_cost:
        print(cycle_dur)
        min_inv_cost = inv_cost
        min_cycle_dur = cycle_dur
        min_offset = offset
    
print(offset)

In [ ]:
min_offset

In [ ]:
min_cycle_dur

In [ ]:
min_inv_cost

In [ ]:
time_horizon

In [ ]:
int(time_horizon)*(0.5*slack_capacity)

In [ ]:
inv_cost

In [ ]:
print(overal_stockout_prob)

In [ ]:
overal_stockout_prob

In [ ]:
probs = []
deficit = 10000
for i in range(deficit):
    v = -offset-i
    mu = slack_capacity*time_horizon
    std = np.sqrt(2*mu_supply*time_horizon)
    probs.append(norm.cdf(v,mu,std))
    #print(i,probs[-1])
    
np.sum(-np.diff(probs)*np.arange(1,deficit)/norm.cdf(-offset,mu,std))

In [ ]:
offsets = []
for dur in range(1,time_horizon+1):
    for offset in range(200):
        stock_out_probs = []
        for d in range(1,dur+1):
            stock_out_prob = norm.cdf(-offset,slack_capacity*d,np.sqrt(2*mu_supply*d))
            stock_out_probs.append(stock_out_prob)
        overal_stockout_prob = np.mean(stock_out_probs)
        #print(overal_stockout_prob)
        if overal_stockout_prob < 0.05:
            break
    #print(dur,offset)
    offsets.append(offset)

In [ ]:
plt.plot(offsets)

In [ ]:
norm.cdf(-offset,mu,std)

In [ ]:
offset

In [ ]:
mu

In [ ]:
(-np.diff(probs)/norm.cdf(-offset,mu,std))[:50]

In [ ]:
-np.diff(probs)/norm.cdf(-offset,mu,std)

In [ ]:
offset

In [ ]:
np.sum(last_day_stockouts)/(n_sims)

In [ ]:
sum(stockouts)/(int(np.ceil(time_horizon))*n_sims)

In [ ]:
np.sum(last_day_stockouts)

In [ ]:
np.sum(last_day_stockouts)/sum(stockouts)

In [ ]:
np.mean(stockouts)

In [ ]:
stockouts = np.array(stockouts)

In [ ]:
np.median(stockouts[stockouts > 0])

In [ ]:
plt.hist(stockouts[stockouts > 0])

In [ ]:
plt.hist(stockouts,bins=range(0,50,2))

In [ ]:
2*time_horizon

In [ ]:
norm.cdf(-offset,slack_capacity*10,np.sqrt(mu_supply*10))

In [ ]:
int(np.ceil(time_horizon))